In [1]:
import sys, os
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
SRC_DIR = os.path.join(BASE_DIR, "src")
sys.path.append(SRC_DIR)

In [2]:
import matplotlib.pyplot as plt
import random
import json 
from reporting.explore_stats_sumry import summarize_dataset_exclusive, classify_doc
from utils.io_utils import ensure_dir, read_jsonl, sample_jsonl
from reporting.explore_stats_sumry import quick_stats, show_longest_docs, summarize_dataset, summarize_dataset_exclusive


# Read clean file

In [3]:
clean_data_path='../data/clean'
clean_file=os.path.join(clean_data_path, "mainpipe_clean_data2.jsonl")

In [4]:
summary, summary_pct = summarize_dataset_exclusive(clean_file, sample_size=25000)
print(summary)
print(summary_pct)

Counter({'GOOD_ENGLISH': 13796, 'CODE_HEAVY': 11127, 'HTML': 56, 'SHORT_ENGLISH': 21})
{'CODE_HEAVY': 44.507999999999996, 'GOOD_ENGLISH': 55.184, 'HTML': 0.22399999999999998, 'SHORT_ENGLISH': 0.084}


In [5]:
quick_stats(clean_file, sample_size=25000)

FILE OVERVIEW
File size: 199.26 MB (0.19 GB)
Total lines in file: 107,320
Sampled (limit): 25,000
------------

KEYS
Most common keys: [('text', 25000)]
------------

TEXT LENGTH STATS
Avg length: 1823.19
Median length: 840.00
10th percentile: 281.00
90th percentile: 3950.10
Most entries are multi-paragraph documents (median text size>300).
Good for LLM pretraining (rich context, natural text).
Max length: 49579
raw HTML pages,  code dumps / stack traces, JSON logs or config files, full chat transcripts, base64 or encoded junk.
 These must be  filtered or trim these before LLM pretraining
Min length: 120
------------

NOISE INDICATORS
Empty texts: 0 (0.00%)
Short < 100 chars: 0 (0.00%)
Only a small fraction of trivial/low-value text.  Dataset likely contains substantial natural text, not noise
------------

Done.


In [6]:
for line in random.sample(list(open(clean_file)), 20):
    row = json.loads(line)
    text = row["text"]
    if classify_doc(text) == "CODE_HEAVY":  #HTML CODE_HEAVY
        print(text[:500])
        print("-----")


Where did all my fitness motivation go? A few years back, I was so pumped to exercise and go to the gym. I always made sure I never miss a workout. It lasted a good while. But, times have changed now. The question I repeatedly ask myself today is this: "Where did all my fitness motivation go?" I don't have that much time to workout anymore. I haven't even ran a single mile for months now. It's crazy. As of this writing, I've gained back all the weight that I lost in the last few years that I hav
-----
def playlist_song(self, playlist_song_id): """Get information about a playlist song. Note: This returns the playlist entry information only. For full song metadata, use :meth:`song` with the ``'trackId'`` field. Parameters: playlist_song_id (str): A playlist song ID. Returns: dict: Playlist song information. """ playlist_song_info = next( ( playlist_song for playlist in self.playlists(include_songs=True) for playlist_song in playlist['tracks'] if playlist_song['id'] == playlist_song_id ),

When using strong code detect, there is 44% code heavy detection. So, when I print code heavy out of 20, 11 are found heavy. When I inspect those, only three 3 look like codes.
This is because after normalization each document (could be paragraph worth text) is one big line, and if in thatbig line there is any code-like simple, you get code_lines / len(lines)=1 which is >0.4. So the whole document is classified as code heavy now.

### my cleaning pipiline is able to clean most docs with code inside text